In [7]:
from pathlib import Path

import numpy as np
import pandas as pd

In [3]:
data_path = Path("data/")

## MNIST

In [9]:
df = pd.read_parquet(data_path / "mnist" / "mnist.parquet")

In [14]:
df["image"] = df["image"].apply(np.array)

In [10]:
df.columns

Index(['class', 'split', 'image'], dtype='object')

## Pets

In [4]:
df = pd.read_parquet(data_path / "pets" / "pets_clip.parquet")

In [5]:
df.columns

Index(['path', 'image', 'clip', 'clip_norm', 'class', 'species', 'image_n'], dtype='object')